In [824]:
import pandas as pd
import numpy as np
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [825]:
start_time = time.time()
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.4940028190612793 seconds ---


In [826]:
start_time = time.time()

#Step2 Split workload_A by creating different dataframes

workload_A_ids = workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = workload_A[workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.10571646690368652 seconds ---


In [827]:
start_time = time.time()

#Step3 Get metrics

tmpframe = workload_A.T.copy()
tmpframe = tmpframe.tail(585-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

print("--- %s seconds ---" % (time.time() - start_time))


--- 2.3197710514068604 seconds ---


In [828]:
#Step4 FA for all workloads together
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize


start_time = time.time()
all_metrics_data = tmpframe.values

all_metrics_data_normalized = normalize(all_metrics_data, norm='max')
all_metrics_data_normalized_Trans = all_metrics_data_normalized.T

#all_metrics_data_Trans = all_metrics_data.T
#all_metrics_data_normalized_Trans = normalize(all_metrics_data_Trans, norm='max')

print(all_metrics_data_Trans.shape)
tmp_all_transformer = FactorAnalysis(n_components=300, random_state=0)
tmp_workload_A_transformed = tmp_all_transformer.fit_transform(all_metrics_data_normalized_Trans)
print(tmp_workload_A_transformed.shape)
print("--- %s seconds ---" % (time.time() - start_time))


(572, 18349)
(572, 300)
--- 15.695021152496338 seconds ---


In [829]:
#Step5 K-means for all workloads together
from sklearn.cluster import KMeans

num_M = 3

workload_A_list_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(tmp_workload_A_transformed)
cluster_centers = workload_A_list_array_Kmeans.cluster_centers_
tmp_centers_index = []
tmp_centers_metrics_name = []
for i in range(num_M):
    tmp_center = cluster_centers[i]
    tmp_center_diff = tmp_center - tmp_workload_A_transformed
    tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
    tmp_center_min_idx = np.argmin(tmp_center_distance)
    tmp_centers_index.append(tmp_center_min_idx)
    
for j in range(num_M):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
print(tmp_centers_index)
print(tmp_centers_metrics_name)

workload_list_centers_metrics_name = []
for n in range(58):
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)

[200, 90, 126]
['executor.jvmGCTime.avg', 'driver.jvm.pools.PS-Eden-Space.committed.avg', 'executor.cpuTime.avg']


In [830]:
quick_view_metric = workload_A[tmp_centers_metrics_name]
quick_view_metric

,executor.jvmGCTime.avg,driver.jvm.pools.PS-Eden-Space.committed.avg,executor.cpuTime.avg
0,2871.972673,1.657668e+09,1.826837e+11
1,5305.223214,1.725497e+09,6.162674e+10
2,3218.352083,1.583612e+09,6.299830e+10
3,2586.837054,1.577189e+09,5.983338e+10
4,2244.046875,1.357316e+09,1.563849e+11
...,...,...,...
18344,13871.422595,1.731723e+09,3.884193e+11
18345,16487.433314,1.703937e+09,4.016819e+11
18346,13037.349878,1.186276e+09,6.326932e+11
18347,34443.626830,1.768948e+09,1.757525e+12


In [831]:
'''start_time = time.time()

#Step4 FA (apply for all workloads)
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize

#transformer = FactorAnalysis(n_components=7, random_state=0)
workload_list_afterFA = []

for i in range(58):
    tmp_workload_A_listformetrics = workload_A_listformetrics[i]
    tmp_workload_A_listformetrics_array = tmp_workload_A_listformetrics.values
    #print(tmp_workload_A_listformetrics)
    tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array, norm='l2')
    tmp_workload_A_listformetrics_array_normalized_Trans = tmp_workload_A_listformetrics_array_normalized.T
    
    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T
    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T
    #tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm='l2')
    tmp_transformer = FactorAnalysis(n_components=20, random_state=0)
    #print(tmp_workload_A_listformetrics_array_Trans)
    #normalize tmp_workload_A_listformetrics_array_Trans
    #tmp_workload_A_listformetrics_array_Trans_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm='l2')
    #print('-------------------------')
    #print(tmp_workload_A_listformetrics_array_Trans_normalized)
    
    tmp_workload_A_listformetrics_array_Trans_transformed = tmp_transformer.fit_transform(tmp_workload_A_listformetrics_array_normalized_Trans)
    workload_list_afterFA.append(tmp_workload_A_listformetrics_array_Trans_transformed)

print("--- %s seconds ---" % (time.time() - start_time))
#print(workload_list_afterFA[0].shape)
'''

'start_time = time.time()\n\n#Step4 FA (apply for all workloads)\nfrom sklearn.decomposition import FactorAnalysis\nfrom sklearn.preprocessing import normalize\n\n#transformer = FactorAnalysis(n_components=7, random_state=0)\nworkload_list_afterFA = []\n\nfor i in range(58):\n    tmp_workload_A_listformetrics = workload_A_listformetrics[i]\n    tmp_workload_A_listformetrics_array = tmp_workload_A_listformetrics.values\n    #print(tmp_workload_A_listformetrics)\n    tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array, norm=\'l2\')\n    tmp_workload_A_listformetrics_array_normalized_Trans = tmp_workload_A_listformetrics_array_normalized.T\n    \n    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T\n    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T\n    #tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm=\'l2\')\n    tmp_

In [832]:
'''start_time = time.time()

#Step5 K-means (apply for all workloads)
from sklearn.cluster import KMeans

num_M = 2

workload_list_centers_metrics_name = []

for i in range(58):
    tmp_workload_A_listformetrics_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(workload_list_afterFA[i])
    tmp_cluster_centers = tmp_workload_A_listformetrics_array_Kmeans.cluster_centers_
    #print(tmp_cluster_centers)
    tmp_centers_index = []
    tmp_centers_metrics_name= []
    #print(workload_list_afterFA[i].shape)
    for j in range(num_M):
        tmp_center = tmp_cluster_centers[j]
        tmp_center_diff = tmp_center - workload_list_afterFA[i]
        tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
        tmp_center_min_idx = np.argmin(tmp_center_distance)
        tmp_centers_index.append(tmp_center_min_idx)
        #print(tmp_center_min_idx)
        #print('-----')
    #tmp_centers_metrics_name = []
    tmp_n_centers = len(tmp_cluster_centers)
    #print(tmp_n_centers)
    for j in range(tmp_n_centers):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
    
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)

print("--- %s seconds ---" % (time.time() - start_time))
print(workload_list_centers_metrics_name[0])
'''

'start_time = time.time()\n\n#Step5 K-means (apply for all workloads)\nfrom sklearn.cluster import KMeans\n\nnum_M = 2\n\nworkload_list_centers_metrics_name = []\n\nfor i in range(58):\n    tmp_workload_A_listformetrics_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(workload_list_afterFA[i])\n    tmp_cluster_centers = tmp_workload_A_listformetrics_array_Kmeans.cluster_centers_\n    #print(tmp_cluster_centers)\n    tmp_centers_index = []\n    tmp_centers_metrics_name= []\n    #print(workload_list_afterFA[i].shape)\n    for j in range(num_M):\n        tmp_center = tmp_cluster_centers[j]\n        tmp_center_diff = tmp_center - workload_list_afterFA[i]\n        tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)\n        tmp_center_min_idx = np.argmin(tmp_center_distance)\n        tmp_centers_index.append(tmp_center_min_idx)\n        #print(tmp_center_min_idx)\n        #print(\'-----\')\n    #tmp_centers_metrics_name = []\n    tmp_n_centers = len(tmp_clust

In [833]:
start_time = time.time()

#Step6 Get Prutiend workload (apply for all workloads)

knobs_latency_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']
latency = ['latency']
pruning_workload_list = []

for i in range(num_workloads_A):
    #tmp_pruning_list = knobs_latency_list + latency + tmp_centers_metrics_name
    tmp_pruning_list = knobs_latency_list + latency + workload_list_centers_metrics_name[i]
    tmp_pruning_workload_A = (workload_A_list[i])[tmp_pruning_list]
    pruning_workload_list.append(tmp_pruning_workload_A)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.043883562088012695 seconds ---


In [834]:
start_time = time.time()

#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler


tmp_kernel = 1.0 * RBF(length_scale=1.0)

workload_A_GPRmodels_lists = []
workload_A_scalar_lists = []
workload_A_normalized_m = []

for i in range(58):
    tmp_GPRmodels_lists = []
    tmp_scalar_lists = []
    tmp_normalized_m = []
    
    tmp_Pruning_workload_A_X = ((workload_A_list[i])[knobs_latency_list]).values
    tmp_Pruning_workload_A_y_o = ((workload_A_list[i])[latency]).values
    #print(tmp_Pruning_workload_A_y_o)
    
    tmp_x_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_X)
    tmp_y_o_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_y_o)
    
    tmp_scalar_lists.append(tmp_x_scaler)
    tmp_scalar_lists.append(tmp_y_o_scaler)
    
    tmp_Pruning_workload_A_X_normalized = tmp_x_scaler.transform(tmp_Pruning_workload_A_X)
    #print(tmp_x_scaler.data_min_)
    tmp_Pruning_workload_A_y_o_normalized = tmp_y_o_scaler.transform(tmp_Pruning_workload_A_y_o)
    
    tmp_normalized_m.append(tmp_Pruning_workload_A_y_o_normalized)
    #print(tmp_Pruning_workload_A_y_o_normalized)
    
    #tmp_kernel = DotProduct() + WhiteKernel()
    tmp_workload_A_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X_normalized, tmp_Pruning_workload_A_y_o_normalized)
    
    tmp_GPRmodels_lists.append(tmp_workload_A_gpr_o)
    
    tmp_num_centers = len(workload_list_centers_metrics_name[i])
    for j in range(tmp_num_centers):
        
        tmp_Pruning_workload_A_y_m = ((workload_A_list[i])[[workload_list_centers_metrics_name[i][j]]]).values
        #print(tmp_Pruning_workload_A_y_m)
        tmp_y_m_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_y_m)
        tmp_scalar_lists.append(tmp_y_m_scaler)
        tmp_Pruning_workload_A_y_m_normalized = tmp_y_m_scaler.transform(tmp_Pruning_workload_A_y_m)
        tmp_normalized_m.append(tmp_Pruning_workload_A_y_m_normalized)
        
        #tmp_kernel = DotProduct() + WhiteKernel()
        tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X_normalized, tmp_Pruning_workload_A_y_m_normalized)
        tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
        
    workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)
    workload_A_scalar_lists.append(tmp_scalar_lists)
    workload_A_normalized_m.append(tmp_normalized_m)
print("--- %s seconds ---" % (time.time() - start_time))
#print(workload_A_normalized_m)
print(workload_A_scalar_lists[0])

--- 36.881141662597656 seconds ---
[MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1)), MinMaxScaler(copy=True, feature_range=(0, 1))]


In [835]:
#*************************************************************************************************************************************************
#    1
#    We have workload_A_list contain 58 workloads
#    workload_A_list
#
#    2
#    We have workload_list_centers_metrics_name contain 58 lists, each list have the metrics kept after FA and K-means for each workload
#    workload_list_centers_metrics_name
#
#    3
#    We have pruning_workload_list contain 58 workloads after prunning
#    pruning_workload_list
#
#    4
#    We have workload_A_GPRmodels_lists contain 58 lists, each list have 1(latency) + k(number of metrics kept) GPR models
#    workload_A_GPRmodels_lists
#
#*************************************************************************************************************************************************

In [836]:
workload_list_centers_metrics_name[0]

['executor.jvmGCTime.avg',
 'driver.jvm.pools.PS-Eden-Space.committed.avg',
 'executor.cpuTime.avg']

In [837]:
#Phase2 Using the online workload B as the validation data

#In workloads B, each workload has 6 data points, we use 5 of them apply workload mapping, and use the left 1 as the target workload, predict
#latency and compare it with the actual value in workloads B.

In [838]:
start_time = time.time()

#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.08078289031982422 seconds ---


In [839]:
start_time = time.time()

#Step2 Split workload_B by creating different dataframes

workload_B_ids = workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = workload_B[workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.06981492042541504 seconds ---


In [840]:
start_time = time.time()

#Step3 Split each workload into 5 data points and 1 data point as target

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.03992176055908203 seconds ---


In [841]:
start_time = time.time()

#Step4 workload mapping Test for all workload in workload B
all_distance_list = []
for i in range(100):
    tmp_test_workload_B_mapping = workload_B_mapping_data[i]
    tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_latency_list]).values
    tmp_distance_list = []
    
    for j in range(58):
        tmp_test_workload_A = workload_A_list[j]
        tmp_test_workload_A_metrics = workload_list_centers_metrics_name[j]
        tmp_test_workload_A_gpr_model = workload_A_GPRmodels_lists[j]
        tmp_test_prunning_workload_A = pruning_workload_list[j]
        tmp_test_workload_A_cigs_scalar = workload_A_scalar_lists[j][0]
        tmp_test_workload_A_normalized_m = workload_A_normalized_m[j]
        
        tmp_norm_workload_B_mapping_cfigs = tmp_test_workload_A_cigs_scalar.transform(tmp_test_workload_B_mapping_cfigs)
        #print(tmp_norm_workload_B_mapping_cfigs)
        
        tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_latency_list]).values
        #print(tmp_test_workload_A_gpr_model)
        tmp_corr_metric_values = []
        
        for k in range(5):
            tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
            tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
            if len(tmp_check[0]) == 0:
                tmp_gpr_predic = []
                for v in range(num_M):
                    tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[v+1].predict([tmp_norm_workload_B_mapping_cfigs[k]])[0])
                tmp_corr_metric_values.append(tmp_gpr_predic)
                #print(tmp_gpr_predic)
            
            else:
                tmp_gpr_predic = []
                tmp_row_idx = tmp_check[0][0]
                #print(tmp_row_idx)
                #print('xxxxxxxxxxxxxxxxxxx')
                tmp_test_prunning_workload_A_array = tmp_test_prunning_workload_A.values
                #print(tmp_test_workload_A_normalized_m)
                for n in range(num_M):
                    #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13+n])
                    tmp_gpr_predic.append(tmp_test_workload_A_normalized_m[n+1][tmp_row_idx])
                    
                #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13])
                #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][14])
                tmp_corr_metric_values.append(tmp_gpr_predic)
        
        tmp_test_workload_B_mapping_metrics_norm_value_list = []
        for m in range(num_M):
            tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[[tmp_test_workload_A_metrics[m]]].values
            tmp_test_workload_B_mapping_metrics_nomr_value = workload_A_scalar_lists[j][2+m].transform(tmp_test_workload_B_mapping_metrics_value)
            tmp_test_workload_B_mapping_metrics_norm_value_list.append(tmp_test_workload_B_mapping_metrics_nomr_value)
            
            
        #tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((2,5))
        #tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((2,5))
        #print(tmp_test_workload_B_mapping_metrics_norm_value_list)
        
        #print('---')
        #print(tmp_corr_metric_values)
        #print('xxxxxxxxxxxxxx')
        tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((num_M,5))       
        tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((5,num_M)).T
        #print(tmp_test_workload_B_mapping_metrics_norm_value_arry)
        #print('-----')
        #print(tmp_corr_metric_values_arry)
        
        
        #tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[tmp_test_workload_A_metrics].values
        #print(tmp_test_workload_B_mapping_metrics_value)
        #print('--------')
        tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_corr_metric_values_arry)*(tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_corr_metric_values_arry))
        tmp_distance_list.append(tmp_distance)
     
    all_distance_list.append(tmp_distance_list)


    
all_min_idx_list = []
for i in range(100):
    tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
    all_min_idx_list.append(tmp_idx)

print("--- %s seconds ---" % (time.time() - start_time))


--- 23.87873125076294 seconds ---


In [842]:
start_time = time.time()

#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

predictGPRmodel_list = []
predictScalar_list = []
inputScalar_list = []

for i in range(100):
    tmp_similar_idx = all_min_idx_list[i]
    tmp_similar_workload = pruning_workload_list[tmp_similar_idx]
    tmp_similar_workload_configs = ((pruning_workload_list[tmp_similar_idx])[knobs_latency_list]).values
    tmp_similar_workload_metrics = ((pruning_workload_list[tmp_similar_idx])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    
    tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
    tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
    tmp_workload_B_configs = workload_B_mapping_data[i][knobs_latency_list].values
    #print(tmp_workload_B_configs)
    
    for j in range(5):
        tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
        tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_mapping_metrics_with_similar[j]
        tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
        if len(tmp_check[0]) == 0:
            #print('z')
            tmp_Augmented_pruned_target_configs = np.insert(tmp_Augmented_pruned_target_configs,0,tmp_test_workload_B_mapping_cfigs,axis=0)
            tmp_Augmented_pruned_target_metrics = np.insert(tmp_Augmented_pruned_target_metrics,0,tmp_test_workload_B_mapping_metrics_with_similar,axis=0)
        else:
        
            tmp_row_idx = tmp_check[0][0]
            #print(tmp_row_idx)
            #print('xxxxxxxxxxxxxxxxxxx')
            tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
    tmp_metrics_o_size = tmp_Augmented_pruned_target_metrics.T[0].shape[0]
    tmp_nor_tmp_Augmented_pruned_target_metrics_o = tmp_Augmented_pruned_target_metrics.T[0].reshape((tmp_metrics_o_size,1))
    tmp_o_scalar = MinMaxScaler().fit(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    predictScalar_list.append(tmp_o_scalar)
    
    tmp_Augmented_pruned_target_metrics_nomr = tmp_o_scalar.transform(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    tmp_X_scalar = MinMaxScaler().fit(tmp_Augmented_pruned_target_configs)
    inputScalar_list.append(tmp_X_scalar)
    
    tmp_Augmented_pruned_target_configs_norm = tmp_X_scalar.transform(tmp_Augmented_pruned_target_configs)
    #print(tmp_Augmented_pruned_target_metrics_nomr)
    
    #print(tmp_Augmented_pruned_target_configs)
    #print('------')
    #print(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    test_workload_B_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Augmented_pruned_target_configs_norm, tmp_Augmented_pruned_target_metrics_nomr)
    predictGPRmodel_list.append(test_workload_B_gpr_o)
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 16.334603548049927 seconds ---


In [848]:
start_time = time.time()

#Step6 predict for all workload B
right_count = 0
for i in range(100):
    tmp_target_config = workload_B_target[i][knobs_latency_list].values
    #print(tmp_target_config)
    tmp_target_config_norm = inputScalar_list[i].transform(tmp_target_config)
    #print(tmp_target_config_norm)
    tmp_max = (predictScalar_list[i].data_max_[0])
    tmp_min = (predictScalar_list[i].data_min_[0])
    tmp_predict = predictGPRmodel_list[i].predict(tmp_target_config_norm)
    tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
    tmp_diff = np.abs(tmp_predict_re - workload_B_target[i][latency].values)
    print('Predict: ',tmp_predict_re)
    print('Ground Truth: ',workload_B_target[i][latency].values)
    print('diff: ',tmp_diff)
    if tmp_diff < 3:
        right_count += 1
    print('------------')
    
print('Right Predict: ', right_count)
    
print("--- %s seconds ---" % (time.time() - start_time))


Predict:  [[2.189625]]
Ground Truth:  [[7.923]]
diff:  [[5.733375]]
------------
Predict:  [[261.46899986]]
Ground Truth:  [[260.62625]]
diff:  [[0.84274986]]
------------
Predict:  [[16.43549996]]
Ground Truth:  [[109.0855]]
diff:  [[92.65000004]]
------------
Predict:  [[36.735]]
Ground Truth:  [[61.997]]
diff:  [[25.262]]
------------
Predict:  [[90.59599992]]
Ground Truth:  [[89.78475]]
diff:  [[0.81124992]]
------------
Predict:  [[4.39975]]
Ground Truth:  [[24.4785]]
diff:  [[20.07875]]
------------
Predict:  [[5.87975]]
Ground Truth:  [[4.507125]]
diff:  [[1.372625]]
------------
Predict:  [[18.2167499]]
Ground Truth:  [[107.993]]
diff:  [[89.7762501]]
------------
Predict:  [[285.28199981]]
Ground Truth:  [[42.31425]]
diff:  [[242.96774981]]
------------
Predict:  [[56.61299999]]
Ground Truth:  [[52.674]]
diff:  [[3.93899999]]
------------
Predict:  [[2.168]]
Ground Truth:  [[3.084]]
diff:  [[0.916]]
------------
Predict:  [[9.4355]]
Ground Truth:  [[9.70575]]
diff:  [[0.27025]